In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import pymongo
import xarray as xr

In [2]:
client = MongoClient()
db = client.eddies

In [3]:
def format(LON):
    if LON < 0:
        LON = LON + 360
    else:
        LON = LON
    return LON

In [4]:
rootdir = '/home/cz2397/data/cchdo-ctd/'

In [5]:
station = 0
overlap_start = 0
overlap_end = 0
result_start = []
result_end = []

In [6]:
db.rcs_eddies.create_index([('dat_start', pymongo.ASCENDING), ('loc_start', pymongo.ASCENDING)])

u'dat_start_1_loc_start_1'

In [7]:
db.rcs_eddies.create_index([('dat_end', pymongo.ASCENDING), ('loc_end', pymongo.ASCENDING)])

u'dat_end_1_loc_end_1'

In [8]:
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        if file.endswith('ctd.nc'):
            path = os.path.join(subdir, file)
            DS = xr.open_dataset(path)
            DAT = DS.time.values[0]
            LAT = float(DS.latitude.values[0])
            LON = float(format(DS.longitude.values[0]))
            dat_var = np.timedelta64(4, 'D')
            dat_min = pd.to_datetime(DAT - dat_var)
            dat_max = pd.to_datetime(DAT + dat_var)
            ft_start = {
                'dat_start': {'$gt': dat_min, '$lt': dat_max},
                'loc_start': {'$within': {'$center': [[LON, LAT], 1]}}
            }
            ft_end = {
                'dat_end': {'$gt': dat_min, '$lt': dat_max},
                'loc_end': {'$within': {'$center': [[LON, LAT], 1]}}
            }
            count_start = db.rcs_eddies.find(ft_start).count()
            count_end = db.rcs_eddies.find(ft_end).count()
            station = station + 1
            if count_start != 0:
                overlap_start = overlap_start + 1
                result_start.append(path)
            if count_end != 0:
                overlap_end = overlap_end + 1
                result_end.append(path)

100%|██████████| 115/115 [00:02<00:00, 55.98it/s]


In [9]:
station

33603

In [10]:
overlap_start

0

In [11]:
overlap_end

0

In [12]:
result_start.sort()

In [13]:
result_start

[]

In [14]:
result_end.sort()

In [15]:
result_end

[]